# Working with `Llama-3.2-1B`

### Initialization of the Model Pipeline
*A pipeline is essentially a high-level abstraction function that makes working with models easier.*

- A pipeline is initialized for "text-generation" using the Hugging Face Transformers library.
- Model is specified via `model_id = "meta-llama/Llama-3.2-1B"`

#### What is happening during initialization:
- If not already downloaded, download the model weights
- Loads the relevant tokenizer for the model.
- Configures the PyTorch device mapping:
  - GPU is automatically assigned if available/applicable
  - Model specific parameter: `torch_dtype=torch.bfloat16,`

From here, the `pipe` object is essentially the interface to interact with the `Llama-3.2-1B` model for text generation tasks




In [1]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, TrainerCallback
import numpy as np
import evaluate


model_id = "meta-llama/Llama-3.2-1B"

pipe = pipeline(
    "text-generation", 
    model=model_id, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)


Device set to use mps


### Running text generation
`pipe("The key to life is")` serves as the prompt to the model

#### This pipeline will:
1. Tokenize input prompt
2. Run it through the model to create the output based on the model's learned parameters
3. Decode model's output back into humman readable text


In [2]:
pipe("The key to life is")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'The key to life is to be happy and to have a positive attitude. Positive thinking can help you to achieve anything you want'}]

---
# Fine-tuning the model

### Loading the dataset

In this case, we are using the *Great Gatsby* to train the model.

In [3]:
from datasets import load_dataset

# Loading dataset from hugging face (Great Gatsby txt)
ds = load_dataset("TeacherPuffy/book")

# This line prints out the "train" split where each index is a line number
print(ds["train"][100])


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'text': 'Sometimes she and Miss Baker talked at the same time, unobtrusively and with a playful banter that was never quite chatter, as cool as their white dresses and their impersonal eyes, devoid of all desire. They were here—and they accepted Tom and me, making only a polite effort to entertain or be entertained. They knew that dinner would soon be over, and a little later, the evening too would end and be casually put away. It was a stark contrast to the West, where an evening was rushed from one phase to the next, driven by a continually disappointed anticipation or sheer nervous dread of the moment itself.'}


### Tokenization
We are using a tokenizer to process the text and provide padding as well as a truncation strategy to handle varying sequence lengths. The `map` method is used to apply the preprocessing function over the entire dataset

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id  # Ensure pad_token_id is set

def tokenize_function(examples):
    outputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    outputs["labels"] = outputs["input_ids"].copy()  # Set labels to be identical to input_ids
    return outputs

tokenized_datasets = ds.map(tokenize_function)
print(tokenized_datasets)



DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 111
    })
})


Then, to prepare for training, remove and edit columns that hugging face expects.
Here, the text column is removed, keeping `input_ids`, and `attention_mask`

In [5]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.with_format("torch")
print(tokenized_datasets["train"])


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 111
})


---
# Training the model with PyTorch Trainer

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto")
model.config.pad_token_id = tokenizer.eos_token_id  # Update model configuration
model.resize_token_embeddings(len(tokenizer))

# Contains all hyperparameters
training_args = TrainingArguments(output_dir="test_trainer", num_train_epochs=2)

# Computes and reports metrics during training
metric = evaluate.load("accuracy")

# Calculates accuracy of the predictions
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define a callback to print training loss at the end of each epoch
class LogEpochLossCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        # Filter log history for entries with a loss value
        loss_logs = [log for log in state.log_history if "loss" in log]
        if loss_logs:
            last_log = loss_logs[-1]
            print(f"Epoch {state.epoch:.2f} ended with loss: {last_log['loss']:.4f}")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    compute_metrics=compute_metrics,
)

trainer.add_callback(LogEpochLossCallback)



### Launch training:

In [7]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 